# **Hồi quy tuyến tính - Linear Regression**
- Dựa vào độ tương quan ở file 2_Correlation.ipynb có thể thấy:
    + Điểm toán tương quan mạnh với lý hóa
    + Điểm văn tương quan mạnh với sử địa gdcd
- Do đó ta sẽ dự đoán:
    + Điểm toán: dựa trên điểm môn hóa học _ vật lí
    + Điểm văn : dựa trên điểm môn lịch sử _ địa lý _ GDCD

## Các thư viện cần thiết

In [2]:
import numpy as np
import pandas as pd
import optuna

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report
from sklearn.utils.validation import column_or_1d
from imblearn.over_sampling import RandomOverSampler

In [3]:
score_df = pd.read_csv('new_data.csv')
score_df

,cum,sbd,toan,ngu_van,ngoai_ngu,vat_li,hoa_hoc,sinh_hoc,lich_su,dia_ly,gdcd,phan_loai
0,1,1000001,3.6,5.00,4.0,NaN,NaN,NaN,2.75,6.00,8.75,THPT_KHXH
1,1,1000002,8.4,6.75,7.6,NaN,NaN,NaN,8.50,7.50,8.25,THPT_KHXH
2,1,1000003,5.8,7.50,5.0,NaN,NaN,NaN,7.25,5.50,8.75,THPT_KHXH
3,1,1000004,7.4,7.50,8.6,NaN,NaN,NaN,7.50,6.50,7.50,THPT_KHXH
4,1,1000005,7.2,8.50,9.0,NaN,NaN,NaN,8.00,8.50,8.25,THPT_KHXH
...,...,...,...,...,...,...,...,...,...,...,...,...
995430,64,64006584,8.4,6.75,4.6,NaN,NaN,NaN,6.50,6.75,9.00,THPT_KHXH
995431,64,64006585,5.6,6.50,2.8,NaN,NaN,NaN,6.25,6.75,8.50,THPT_KHXH
995432,64,64006586,5.8,6.00,6.6,NaN,NaN,NaN,7.25,8.00,8.00,THPT_KHXH
995433,64,64006587,7.6,6.75,7.0,NaN,NaN,NaN,8.75,7.25,9.75,THPT_KHXH


Phần này sẽ được dùng cho thuât toán linear regression để dự đoán điểm toán và ngữ văn:
- Đầu tiên, ta tạo pipeline gồm các bước chuẩn hóa, thêm polynomial features và cuối cùng là áp dụng linear regression
- Để có thể chọn được mô hình với tham số tốt nhất, ở đây ta dùng GridSearchCV của 12 candidates và có 10 folds

In [10]:
def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(StandardScaler(), PolynomialFeatures(degree), LinearRegression(**kwargs))

param_grid = {'polynomialfeatures__degree': [3,4,5],
              'linearregression__fit_intercept': [True, False],
              'linearregression__positive': [True, False]}
poly_grid = GridSearchCV(PolynomialRegression(), param_grid=param_grid, cv=10, scoring='neg_mean_squared_error')

## **1. Dự đoán điểm toán: dựa vào hóa học _ vật lí (KHTN)**

#### Pre-process (splitiing data)
Trước tiên ta sẽ tiền xử lí bỏ các giá trị nan và split df ra thành 2 sets: train-test (với tỉ lệ 90-10)

In [11]:
X_toan = score_df[['toan', 'vat_li', 'hoa_hoc']].dropna()[['vat_li', 'hoa_hoc']]
y_toan = score_df[['toan', 'vat_li', 'hoa_hoc']].dropna()[['toan']]

X_train_toan, X_test_toan, y_train_toan, y_test_toan = train_test_split(X_toan, y_toan, test_size=0.1, random_state=0)

#### Trainning model
Sau đó, ta sẽ fit trên grid để tìm bộ tham số tốt nhất

In [12]:
%time poly_grid.fit(X_train_toan, y_train_toan)

CPU times: user 1min 2s, sys: 1.88 s, total: 1min 4s
Wall time: 29.9 s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__fit_intercept': [True, False],
                         'linearregression__positive': [True, False],
                         'polynomialfeatures__degree': [3, 4, 5]},
             scoring='neg_mean_squared_error')

In [13]:
print('Best Score: %s' % poly_grid.best_score_)
print('Best Hyperparameters: %s' % poly_grid.best_params_)

Best Score: -0.5174110217757335
Best Hyperparameters: {'linearregression__fit_intercept': True, 'linearregression__positive': False, 'polynomialfeatures__degree': 5}


- Dựa trên bộ tham số này ta sẽ tạo pipeline tương ứng
- Fit bộ dữ liệu train trên pipeline này và thực hiện dự đoán dựa trên test

In [14]:
pipe_polreg = PolynomialRegression(degree=poly_grid.best_params_['polynomialfeatures__degree'],
                                   fit_intercept=poly_grid.best_params_['linearregression__fit_intercept'],
                                   positive=poly_grid.best_params_['linearregression__positive'])

pipe_polreg.fit(X_train_toan, y_train_toan)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=5)),
                ('linearregression', LinearRegression())])

#### Validate model
Kế tiếp ta validation mô hình trên 10-folds dựa trên MSE

In [15]:
cross_val_score(pipe_polreg, X_train_toan, y_train_toan, cv=10, scoring='neg_mean_squared_error')

array([-0.52328432, -0.51727214, -0.51482147, -0.50991578, -0.51905274,
       -0.52174178, -0.51696104, -0.51369144, -0.51670007, -0.52066943])

#### Testing model
- Cuối cùng, ta sẽ kiểm thử trên bộ dữ liệu test
- Có thể thấy cách chỉ số tính ra đều ổn, cũng như một vài giá trị dự đoán và giá trị thực là tương đối gần bằng nhau

In [16]:
y_test_pred_toan = pipe_polreg.predict(X_test_toan)
print('RMSE: %s' % mean_squared_error(y_test_toan, y_test_pred_toan, squared=False))
print('MSE: %s' % mean_squared_error(y_test_toan, y_test_pred_toan))
print('MAE: %s' % mean_absolute_error(y_test_toan, y_test_pred_toan))
print('Model score:', pipe_polreg.score(X_test_toan, y_test_pred_toan))

RMSE: 0.7145047770164877
MSE: 0.5105170763793808
MAE: 0.5416058503581866
Model score: 1.0


In [23]:
predict_toan_df = X_test_toan.copy()
predict_toan_df['Output'] = y_test_toan
predict_toan_df['Predict'] = y_test_pred_toan
predict_toan_df

,vat_li,hoa_hoc,Output,Predict
446204,7.75,8.00,7.4,8.098424
967293,8.50,5.25,8.4,8.340112
778919,7.00,6.75,7.6,7.639045
659572,6.25,6.50,7.0,7.364588
717937,8.75,7.50,8.6,8.420190
...,...,...,...,...
116898,5.75,4.00,4.2,6.885830
916656,4.00,2.75,6.0,5.736614
132790,7.50,7.25,7.2,7.880957
487097,7.00,8.25,8.6,8.011858


## **2. Dự đoán điểm văn: dựa vào lịch sử _ địa lý _ GDCD (KHXH)**

#### Pre-process (splitiing data)
Trước tiên ta sẽ tiền xử lí bỏ các giá trị nan và split df ra thành 2 sets: train-test (với tỉ lệ 90-10)

In [25]:
X_nv = score_df[['ngu_van', 'lich_su', 'dia_ly', 'gdcd']].dropna()[['lich_su', 'dia_ly', 'gdcd']]
y_nv = score_df[['ngu_van', 'lich_su', 'dia_ly', 'gdcd']].dropna()[['ngu_van']]

X_train_nv, X_test_nv, y_train_nv, y_test_nv = train_test_split(X_nv, y_nv, test_size=0.1, random_state=0)

#### Trainning model
Sau đó, ta sẽ fit trên grid để tìm bộ tham số tốt nhất

In [26]:
%time poly_grid.fit(X_train_nv, y_train_nv)

CPU times: user 4min 12s, sys: 35.9 s, total: 4min 48s
Wall time: 3min 13s


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('polynomialfeatures',
                                        PolynomialFeatures()),
                                       ('linearregression',
                                        LinearRegression())]),
             param_grid={'linearregression__fit_intercept': [True, False],
                         'linearregression__positive': [True, False],
                         'polynomialfeatures__degree': [3, 4, 5]},
             scoring='neg_mean_squared_error')

In [27]:
print('Best Score: %s' % poly_grid.best_score_)
print('Best Hyperparameters: %s' % poly_grid.best_params_)

Best Score: -1.3812864302087862
Best Hyperparameters: {'linearregression__fit_intercept': False, 'linearregression__positive': False, 'polynomialfeatures__degree': 5}


- Dựa trên bộ tham số này ta sẽ tạo pipeline tương ứng
- Fit bộ dữ liệu train trên pipeline này và thực hiện dự đoán dựa trên test

In [28]:
pipe_polreg = PolynomialRegression(degree=poly_grid.best_params_['polynomialfeatures__degree'],
                                   fit_intercept=poly_grid.best_params_['linearregression__fit_intercept'],
                                   positive=poly_grid.best_params_['linearregression__positive'])

pipe_polreg.fit(X_train_nv, y_train_nv)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures(degree=5)),
                ('linearregression', LinearRegression(fit_intercept=False))])

#### Validate model
Kế tiếp ta validation mô hình trên 10-folds dựa trên MSE

In [29]:
cross_val_score(pipe_polreg, X_train_nv, y_train_nv, cv=10, scoring='neg_mean_squared_error')

array([-1.39841236, -1.37317648, -1.37067256, -1.3728315 , -1.39088717,
       -1.38041596, -1.37085945, -1.38329436, -1.38541101, -1.38690344])

#### Testing model
- Cuối cùng, ta sẽ kiểm thử trên bộ dữ liệu test
- Có thể thấy cách chỉ số tính ra đều khá ổn, cũng như một vài giá trị dự đoán và giá trị thực là tương đối gần bằng nhau
- Tuy vậy lỗi bình phương trung bình vẫn lớn hơn mô hình ở phần 1

In [30]:
y_test_pred_nv = pipe_polreg.predict(X_test_nv)
print('RMSE: %s' % mean_squared_error(y_test_nv, y_test_pred_nv, squared=False))
print('MSE: %s' % mean_squared_error(y_test_nv, y_test_pred_nv))
print('MAE: %s' % mean_absolute_error(y_test_nv, y_test_pred_nv))
print('Model score:', pipe_polreg.score(X_test_nv, y_test_pred_nv))

RMSE: 1.1818903177371458
MSE: 1.3968647231608113
MAE: 0.948635548933555
Model score: 1.0


In [34]:
predict_nv_df = X_test_nv.copy()
predict_nv_df['Output'] = y_test_nv
predict_nv_df['Predict'] = y_test_pred_nv
predict_nv_df

,lich_su,dia_ly,gdcd,Output,Predict
186176,4.50,4.25,5.25,3.25,4.810653
222803,6.50,5.25,8.75,4.50,6.708959
616110,7.25,8.00,8.00,6.75,6.940266
46556,6.75,7.75,8.00,7.50,6.792566
487931,5.00,5.00,8.25,4.50,6.235406
...,...,...,...,...,...
617138,4.00,5.00,6.25,5.50,5.321021
864566,7.25,8.50,8.50,8.00,7.214333
747887,5.50,6.50,8.50,6.25,6.586211
954464,6.00,6.50,8.00,5.00,6.524992


# **Logistics Regression**
- Dựa vào sự phân bố loại tốt nghiệp ở file 1_Distribution.ipynb có thể thấy:
    + Tỉ lệ học sinh rớt tốt nghiệp rất ít (3%)
    + Nên không đủ để đánh giá, dự đoán xem thí sinh đậu hay rớt
- Với file 2_Correlation:
    + Môn toán (KHTN):
        + Tương quan mạnh với vật lí _ hóa học
        + Tương quan vừa với ngoại ngữ _ sinh học
    + Môn ngữ văn (KHXH):
        + Tương quan mạnh với ngoại ngữ _ GDCD
        + Tương quan mạnh với lịch sử _Địa lý
- ***Vậy từ điểm môn toán - ngữ văn (Hai môn thi đầu tiên) ta sẽ dự đoán thí sinh tốt nghiệp THPT loại "khá - giỏi" hay là không??***

#### Pre-process (create new columns and splitting data)
Trước tiên ta sẽ tiền xử lí để:
- Tính cột điểm tốt nghiệp và từ đó tạo được cột xếp lại tốt nghiệp (đây là code lấy lại của file *1_Distribution.ipynb*)
- Vì phân loại khá chiếm tỉ trọng rất nhiều so với giỏi (gần gắp 8 lần) nên ở đây ta sử dụng random over sampler để đưa dữ liệu không cân bằng về cân bằng
- Lấy ra những dòng có xếp loại tốt nghiệp là giỏi hoặc khá, gán nhãn cho 'Kha' là 0 và 'Gioi' là 1, loại bỏ những giá trị nan và split df ra thành 2 sets: train-test (với tỉ lệ 90-10)

In [4]:
score_df['diem_tn'] = score_df.loc[(score_df['phan_loai'] != 'DH_RETEST') & (score_df['phan_loai'] != 'TH_RETEST'),
            ['toan','ngu_van', 'ngoai_ngu','vat_li','hoa_hoc','sinh_hoc','lich_su', 'dia_ly', 'gdcd']].mean(axis=1)

def classifier(row):
    if (pd.isna(row['diem_tn'])):
        return np.nan
    elif (min(row['toan'], row['ngu_van'], row['ngoai_ngu'], row['vat_li'], row['hoa_hoc'], row['sinh_hoc'], row['lich_su'], row['dia_ly'], row['gdcd']) < 1):
        return 'Rot'
    elif (row['diem_tn'] >= 8):
        return 'Gioi'
    elif (row['diem_tn'] >= 6.5):
        return 'Kha'
    elif (row['diem_tn'] >= 4.5):
        return 'Trung binh'
    else: return 'Rot'

score_df['xep_loai_tn'] = score_df.apply(classifier, axis = 1)

In [5]:
X_xltn = score_df[(score_df['xep_loai_tn'] == 'Gioi') | (score_df['xep_loai_tn'] == 'Kha')][['toan', 'ngu_van']].dropna()
y_xltn = score_df[(score_df['xep_loai_tn'] == 'Gioi') | (score_df['xep_loai_tn'] == 'Kha')][['xep_loai_tn']]

ros = RandomOverSampler(random_state=0)
X_xltn_resampled, y_xltn_resampled = ros.fit_resample(X_xltn, y_xltn)

X_train_xltn, X_test_xltn, y_train_xltn, y_test_xltn = train_test_split(X_xltn_resampled, y_xltn_resampled, test_size=0.1, random_state=0)
y_train_xltn = column_or_1d(y_train_xltn.replace(['Kha', 'Gioi'], [0, 1]), warn=False)
y_test_xltn = column_or_1d(y_test_xltn.replace(['Kha', 'Gioi'], [0, 1]), warn=False)

#### Trainning model
Sau đó, ta tìm bộ tham số tốt nhất sử dụng optuna trên pipeline (gồm chuẩn hóa StandardScaler và LogisticRegression) dựa trên cross_val_score (có cv là RepeatedStratifiedKFold với k = 10 và lặp lại 3 lần, đánh giá bằng trung bình của f1-score)

In [6]:
%%time
def LogisticRegression_Pipe(**kwargs):
    return make_pipeline(StandardScaler(), LogisticRegression(**kwargs))

def objective(trial):
    penalty = trial.suggest_categorical('penalty', ['l2'])
    tol = trial.suggest_loguniform('tol', 1e-6, 1e-3)
    max_iter = trial.suggest_discrete_uniform('max_iter', 50, 200, 50)
#     solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'])
    fit_intercept = trial.suggest_int('fit_intercept', False, True)

    pipe_logisreg = LogisticRegression_Pipe(penalty=penalty,
                                            tol=tol,
                                            max_iter=max_iter,
#                                             solver=solver,
                                            fit_intercept=fit_intercept)
    
    rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=0)
    return cross_val_score(pipe_logisreg, X_train_xltn, y_train_xltn, cv=rskf, scoring='f1', n_jobs=-1).mean()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

trial = study.best_trial
print('Accuracy: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-01-13 03:02:15,468] Finished trial#0 resulted in value: 0.7918179266959708. Current best value is 0.7918179266959708 with parameters: {'penalty': 'l2', 'tol': 1.7571855365552842e-06, 'max_iter': 200.0, 'fit_intercept': 0}.
[I 2023-01-13 03:02:22,159] Finished trial#1 resulted in value: 0.7918179266959708. Current best value is 0.7918179266959708 with parameters: {'penalty': 'l2', 'tol': 1.7571855365552842e-06, 'max_iter': 200.0, 'fit_intercept': 0}.
[I 2023-01-13 03:02:27,787] Finished trial#2 resulted in value: 0.7869792738469561. Current best value is 0.7918179266959708 with parameters: {'penalty': 'l2', 'tol': 1.7571855365552842e-06, 'max_iter': 200.0, 'fit_intercept': 0}.
[I 2023-01-13 03:02:34,258] Finished trial#3 resulted in value: 0.7869792738469561. Current best value is 0.7918179266959708 with parameters: {'penalty': 'l2', 'tol': 1.7571855365552842e-06, 'max_iter': 200.0, 'fit_intercept': 0}.
[I 2023-01-13 03:02:39,048] Finished trial#4 resulted in value: 0.7918179266

Accuracy: 0.7918179266959708
Best hyperparameters: {'penalty': 'l2', 'tol': 1.7571855365552842e-06, 'max_iter': 200.0, 'fit_intercept': 0}
CPU times: user 28 s, sys: 6.68 s, total: 34.7 s
Wall time: 1min 31s


- Dựa trên bộ tham số này ta sẽ tạo pipeline tương ứng
- Fit bộ dữ liệu train trên pipeline này và thực hiện dự đoán dựa trên test

In [7]:
pipe_logisreg = LogisticRegression_Pipe(penalty=trial.params['penalty'],
                                        tol=trial.params['tol'],
                                        max_iter=trial.params['max_iter'],
#                                         solver=trial.params['solver'],
                                        fit_intercept=trial.params['fit_intercept'])

pipe_logisreg.fit(X_train_xltn, column_or_1d(y_train_xltn, warn=False))

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression',
                 LogisticRegression(fit_intercept=0, max_iter=200.0,
                                    tol=1.7571855365552842e-06))])

#### Validate model
Kế tiếp ta validation mô hình trên 10-folds dựa trên f1-score

In [8]:
cross_val_score(pipe_logisreg, X_train_xltn, column_or_1d(y_train_xltn, warn=False), cv=10, scoring='f1')

array([0.7918579 , 0.79042315, 0.789952  , 0.79396068, 0.79322485,
       0.79316225, 0.7901317 , 0.79214861, 0.79366312, 0.78965656])

#### Testing model
- Cuối cùng, ta sẽ kiểm thử trên bộ dữ liệu test
- Nhờ vào việc sử dụng oversampling mà để xử lí việc khá gấp 8 lần giỏi mô hình đã cho ra f1-score cho cả hai lớp 0 và 1 là gần như nhau (nếu không có oversampling thì f1-score của khá sẽ cao và giỏi sẽ rất thấp)
- Xét 4 chỉ số accuracy, precision, recall, f1-score cho cả hai lớp 0 và 1 đều khá ổn

In [9]:
y_test_pred_xltn = pipe_logisreg.predict(X_test_xltn)
print(classification_report(y_test_xltn, y_test_pred_xltn))
y_test_pred_xltn

              precision    recall  f1-score   support

           0       0.81      0.74      0.77     38085
           1       0.76      0.83      0.80     38475

    accuracy                           0.78     76560
   macro avg       0.79      0.78      0.78     76560
weighted avg       0.79      0.78      0.78     76560



array([0, 1, 0, ..., 0, 1, 1])

In [36]:
predict_xltn_df = X_test_xltn.copy()
predict_xltn_df['Output'] = y_test_xltn
predict_xltn_df['Predict'] = y_test_pred_xltn
predict_xltn_df

,toan,ngu_van,Output,Predict
297814,7.4,6.25,0,0
442526,8.4,8.75,1,1
363585,7.6,6.75,0,0
501353,7.0,7.50,1,0
181275,7.0,9.00,0,1
...,...,...,...,...
48920,8.6,6.25,0,0
417611,8.2,8.25,0,1
384062,7.8,7.25,0,0
225309,8.4,8.50,1,1
